In [1]:
import requests
import numpy as np
import librosa
import os
import pytube
from moviepy.video.io.VideoFileClip import VideoFileClip

In [2]:
def download_youtube_audio(url, output_path):
    # Download the video as an MP4 file
    youtube = pytube.YouTube(url)
    video = youtube.streams.filter(only_audio=True).first().download()

    # Convert the MP4 file to an MP3 file
    mp4_file = VideoFileClip(video)
    mp4_file.audio.write_audiofile(output_path, fps=44100, nbytes=2, codec='pcm_s16le')
    mp4_file.close()
    os.remove(video)

    print(f"Downloaded and converted audio to {output_path}")

In [3]:
download_youtube_audio('https://www.youtube.com/watch?v=U41bONK2V-U', '/Users/marcowhite/Documents/vortex/source_separation.mp3')
#download_youtube_audio('https://www.youtube.com/watch?v=G2IhFFars4o', '/Users/marcowhite/Documents/vortex/source_separation.mp3')

KeyError: 'video_fps'

In [ ]:
def get_visualization_data(audio_file_path):
    # Load audio file
    audio_data, sr = librosa.load(audio_file_path, sr=44100, mono=True)

    # Submit audio data to Open-Unmix API
    payload = {
        'file': ('audio.wav', audio_data.tobytes(), 'audio/wav'),
        'parameters': ('', '{"separator": "spleeter"}')
    }
    response = requests.post('https://www.aicrowd.com/evaluations/generate_mask', files=payload)

    # Parse response
    mask_data = np.frombuffer(response.content, dtype=np.float32)
    mask_data = np.reshape(mask_data, (5, len(audio_data)))

    # Construct custom data object
    visualization_data = {}
    for i, instrument_name in enumerate(['drums', 'bass', 'vocals', 'other', 'piano']):
        instrument_mask = mask_data[i]
        instrument_audio = audio_data * instrument_mask
        instrument_led_colors = []
        for j in range(len(instrument_audio)):
            if instrument_mask[j] > 0.1:
                # Set LED color based on amplitude of instrument audio
                led_color = [int(val*255) for val in np.abs(instrument_audio[j]) * 60]
                instrument_led_colors.append(led_color)
            else:
                # Set LED color to off
                instrument_led_colors.append([0, 0, 0])
        visualization_data[instrument_name] = instrument_led_colors

    return visualization_data

In [8]:
download_youtube_audio('https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'rick_astley.mp3')

KeyError: 'video_fps'

In [5]:
from moviepy.editor import VideoFileClip

input_file = "/Users/marcowhite/Documents/vortex/source_separation/DROELOE - Panorama (Official Audio).mp4"
output_file = "/Users/marcowhite/Documents/vortex/source_separation/panorama.wav"

# Extract audio from the MP4 file
clip = VideoFileClip(input_file)
audio = clip.audio

# Write audio to WAV file
audio.write_audiofile(output_file)

# Close audio file
audio.close()

OSError: MoviePy error: the file /Users/marcowhite/Documents/vortex/source_separation.mp4 could not be found!
Please check that you entered the correct path.

In [6]:
from moviepy.editor import VideoFileClip

input_file = "/Users/marcowhite/Documents/vortex/source_separation/DROELOE - Panorama (Official Audio).mp4"
output_file = "/Users/marcowhite/Documents/vortex/source_separation/panorama.wav"

# Extract audio from the MP4 file
clip = VideoFileClip(input_file, fps_source='fps')
audio = clip.audio

# Write audio to WAV file
audio.write_audiofile(output_file)

print(f"Converted video to audio file: {output_file}")


KeyError: 'video_fps'

In [7]:
def get_visualization_data(audio_file_path):
    # Load audio file
    audio_data, sr = librosa.load(audio_file_path, sr=44100, mono=True)

    # Submit audio data to Open-Unmix API
    payload = {
        'file': ('audio.wav', audio_data.tobytes(), 'audio/wav'),
        'parameters': ('', '{"separator": "spleeter"}')
    }
    response = requests.post('https://www.aicrowd.com/evaluations/generate_mask', files=payload)

    # Parse response
    mask_data = np.frombuffer(response.content, dtype=np.float32)
    mask_data = np.reshape(mask_data, (5, len(audio_data)))

    # Construct custom data object
    visualization_data = {}
    for i, instrument_name in enumerate(['drums', 'bass', 'vocals', 'other', 'piano']):
        instrument_mask = mask_data[i]
        instrument_audio = audio_data * instrument_mask
        instrument_led_colors = []
        for j in range(len(instrument_audio)):
            if instrument_mask[j] > 0.1:
                # Set LED color based on amplitude of instrument audio
                led_color = [int(val*255) for val in np.abs(instrument_audio[j]) * 60]
                instrument_led_colors.append(led_color)
            else:
                # Set LED color to off
                instrument_led_colors.append([0, 0, 0])
        visualization_data[instrument_name] = instrument_led_colors

    return visualization_data


In [10]:
audio_path = "/Users/marcowhite/Documents/vortex/source_separation/panorama.mp3"

vis_data = get_visualization_data(audio_path)

ValueError: cannot reshape array of size 0 into shape (5,10129408)